# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> Data Science 2: Advanced Topics in Data Science
## Homework 5 Part 1: Fine Tuning Llama3


**Harvard University**<br/>
**Spring 2025**<br/>
**Instructors**: Pavlos Protopapas, Natesh Pillai, and Chris Gumb


<hr style="height:2pt">

In [1]:
# RUN THIS CELL
import requests
from IPython.core.display import HTML
styles = requests.get(
    "https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/"
    "content/styles/cs109.css"
).text
HTML(styles)

In [ ]:
%pip install torch

import torch
import torch.nn as nn
import pandas as pd
import numpy as np

from datasets import Dataset
from scipy.special import softmax
from sklearn.preprocessing import LabelEncoder
from transformers import (
    BitsAndBytesConfig,
    LlamaPreTrainedModel,
    LlamaModel,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    set_seed
)
from transformers import LlamaPreTrainedModel, LlamaModel
from transformers.modeling_outputs import SequenceClassifierOutput
from huggingface_hub import notebook_login, login
import torch
import torch.nn as nn

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from matplotlib import pyplot as plt

import time
import random
import os
import json
import gc
from collections import namedtuple

def setup_seed(seed):
    '''Sets the seed of the entire notebook so results are the same every time we run.'''
    random.seed(seed)
    set_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)

setup_seed(42)

   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
    --------------------------------------- 2.9/204.1 MB 14.0 MB/s eta 0:00:15
    --------------------------------------- 4.7/204.1 MB 13.6 MB/s eta 0:00:15
   - -------------------------------------- 7.3/204.1 MB 11.9 MB/s eta 0:00:17
   - -------------------------------------- 9.2/204.1 MB 11.0 MB/s eta 0:00:18
   -- ------------------------------------- 10.7/204.1 MB 10.2 MB/s eta 0:00:20
   -- ------------------------------------- 12.1/204.1 MB 9.7 MB/s eta 0:00:20
   -- ------------------------------------- 13.4/204.1 MB 9.1 MB/s eta 0:00:21
   -- ------------------------------------- 14.4/204.1 MB 8.6 MB/s eta 0:00:22
   --- ------------------------------------ 15.5/204.1 MB 8.3 MB/s eta 0:00:23
   --- ------------------------------------ 16.5/204.1 MB 8.2 MB/s eta 0:00:23
   --- ------------------------------------ 17.0/204.1 MB 7.6 MB/s eta 0:00:25
   --- ------------------------------------ 17.8/204.1 MB 7

ModuleNotFoundError: No module named 'datasets'

In [5]:
# Some PyTorch settings
print(f"PyTorch version: {torch.__version__}")
print(f"GPU availablity: \n{torch.cuda.is_available()}\n")

TRAIN_CSV = "data/chatbot_arena_conversations.csv"
model_path = "meta-llama/Llama-3.2-3B-Instruct"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LENGTH = 1024

PyTorch version: 2.1.0.post303
GPU availablity: 
True



In [6]:
# measure notebook runtime
time_start = time.time()

<div style = "background: lightsalmon; border: thin solid black; border-radius: 2px; padding: 5px">

### Instructions
- To submit your notebook, follow the instructions given in on the Canvas assignment page.
- Plots should be legible and interpretable *without having to refer to the code that generated them*. They should includelabels for the $x$- and $y$-axes as well as a descriptive title and/or legend when appropriate.
- When asked to interpret a visualization, do not simply describe it (e.g., "the curve has a steep slope up"), but instead explain what you believe the plot *means*.
- Autograding tests are mostly to help you debug. The tests are not exhaustive so simply passing all tests may not be sufficient for full credit.
- The use of *extremely* inefficient or error-prone code (e.g., copy-pasting nearly identical commands rather than looping) may result in only partial credit.
- We have tried to include all the libraries you may need to do the assignment in the imports cell provided below. Please get course staff approval before importing any additional 3rd party libraries.
- Enable scrolling output on cells with very long output.
- Feel free to add additional code or markdown cells as needed.
- Ensure your code runs top to bottom without error and passes all tests by restarting the kernel and running all cells (note that this can take a few minutes).
- **You should do a "Restart Kernel and Run All Cells" before submitting to ensure (1) your notebook actually runs and (2) all output is visible**
</div>


<a id="contents"></a>

## Notebook Contents

- [**PART 1: Fine Tuning LLM**](#part1)


## About this Homework

In this homework, we will explore how to fine-tune an LLM for a classification task. In particular we will be fine tuning a [Llama3.2 3B model](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct)

- In [PART 1](#part1), we will begin by building a classifier for [Chatbot Arena Conversations dataset](https://huggingface.co/datasets/lmsys/chatbot_arena_conversations)


**IMPORTANT NOTES:**

- LLMs are insanely computationally intensive.
- **We highly recommend that you train your model on a system using GPUs. For this, we recommend using the [GPU-enabled Jupyter environment](https://ood.huit.harvard.edu/pun/sys/dashboard/batch_connect/sys/ood-jupyterlab-spack-conda/cs1090b/session_contexts/new) provided to you as part of this course.**
- Models that take hours to train on CPUs can be trained in just minutes when using GPUs.
- **To avoid getting frustrated by having to re-train your models every time you run your notebook, you should save your trained model weights for later use.** Model history dictionaries can also be saved to disk with `pickle` and checked with an `if not` condition. This is a great way to check if the model weights exist before training, preventing redundant retraining. Please, think of the penguins! 🐧

**KERNEL CRASHES:**

If your kernel crashes as you attempt to train your model, please check the following items:
- Models with too many parameters might not fit in GPU memory. Try reducing the size of your model.
- A large `batch_size` will attempt to load too many samples in GPU memory. Avoid using a very large batch size.
- Avoid creating multiple copies of the data.



<a id="part1"></a>
    
<!-- <div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA"> -->

# PART 1: Fine-Tuning LLMs (50 Points)


<a id="part1intro"></a>

## Overview

[Return to contents](#contents)

In this question, you will work with the [Chatbot Arena Conversations dataset](https://huggingface.co/datasets/lmsys/chatbot_arena_conversations) to build a preference prediction model. Your task is to predict which responses users will prefer when presented with two competing answers generated by different large language models (LLMs). The dataset contains a curated subset of conversations from the [Chatbot Arena platform](https://lmarena.ai/), where human users evaluate responses from various LLMs in head-to-head comparisons. You will fine tune Llama 3.2 3B instruct model that can analyze the user's prompt and the generated responses to determine which response users are more likely to prefer.

Learn more about Chatbot Arena here -> [https://arxiv.org/abs/2403.04132](https://arxiv.org/abs/2403.04132)

[Llama 3.2 Models](https://ai.meta.com/blog/llama-3-2-connect-2024-vision-edge-mobile-devices//)

**Note:** The dataset contains conversations that may be unsafe, offensive, or upsetting.

<div class="alert alert-success" style="color: #333; background-color: #e8fffb; border-color: #bcfff2; border-width: 1px; border-radius: 3px; padding: 10px;">

<a id="q11"></a>

<b>1.1 Initial Preprocessing</b>
<hr>

<b>Q1.1.1 - Preprocessing</b>

<a id="q111"></a>

- We provide the code for this part. We will use a separate script for this part - which will be executed just once.
```python
%%writefile dataset_format.py
## preprocessing code here
```
- `dataset_fomat.py` script does following - 

- Loads [Chatbot Arena Conversations dataset](https://huggingface.co/datasets/lmsys/chatbot_arena_conversations) using [HuggingFace Dataset](https://huggingface.co/docs/datasets/v3.4.1/en/package_reference/loading_methods#datasets.load_dataset)
- Converts dataset to pandas dataframe
- Filters dataset for English language only
- Separates conversations into - prompt, response_a (this is response from model_a) and response_b (this is response from model_b). Take a closer look at prompts, response_a and response_b - they are all lists, some lists have multiple prompts - these are called multi turn prompts.
- To simplify the problem at hand, we drop all rows where there are no clear winners i.e drop rows with winner=='tie'.  Dataset contains much fewer samples where winner==tie.
- Keeps only single turn conversations. Multiturn conversations can be very long and much harder to predict correctly.
- Apply `utf-8` encoding/decoding for consistency and to avoid any potential tokenization errors.

</div>


In [7]:
%%writefile dataset_format.py

import numpy as np
import pandas as pd
import json

from tqdm.auto import tqdm
tqdm.pandas()

# pip install datasets
from datasets import load_dataset

# Load the chatbot arena conversations dataset from HuggingFace
dataset = load_dataset("lmsys/chatbot_arena_conversations", token=True)
print(dataset)
# Convert the dataset to a pandas DataFrame
df = dataset["train"].to_pandas()
df = df[df.language=='English'].reset_index(drop=True)
print(df.shape)

def separate_conversations(conv):
    user_texts      = [x['content'] for x in conv if x['role'] == 'user']
    assistant_texts = [x['content'] for x in conv if x['role'] == 'assistant']

    return user_texts, json.dumps(assistant_texts)


df['prompt_a'], df['response_a'] = zip(*df.conversation_a.progress_apply(separate_conversations))
df['prompt_b'], df['response_b'] = zip(*df.conversation_b.progress_apply(separate_conversations))


assert (df['prompt_a'] == df['prompt_b']).all() == True

df['prompt'] = df['prompt_a'].progress_apply(json.dumps)

def one_hot_encode(winner):
    return pd.Series([int('model_a'==winner), int('model_b'==winner), int('tie'==winner or 'tie (bothbad)'==winner)])

df[['winner_model_a', 'winner_model_b', 'winner_tie']] = df.winner.progress_apply(one_hot_encode)

cols = ['question_id', 'model_a', 'model_b', 'prompt', 'response_a',
        'response_b', 'winner_model_a', 'winner_model_b', 'winner_tie']

df = pd.DataFrame(df[cols].copy().values, columns=cols).reset_index(drop=True)

## Remove ties
df = df[df.winner_tie==0].reset_index(drop=True)
df.drop(columns=['winner_tie'], inplace=True)

df['id'] = df.index

df['prompt_length'] = df['prompt'].apply(lambda x: len(eval(x)))
df['response_a_length'] = df['response_a'].apply(lambda x: len(eval(x)))
df['response_b_length'] = df['response_b'].apply(lambda x: len(eval(x)))

df = df[df['prompt_length'] == 1].reset_index(drop=True)

df['prompt'] = df['prompt'].apply(lambda x: eval(x)[0])
df['response_a'] = df['response_a'].apply(lambda x: eval(x)[0])
df['response_b'] = df['response_b'].apply(lambda x: eval(x)[0])

df['prompt'] = df['prompt'].apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))
df['response_a'] = df['response_a'].apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))
df['response_b'] = df['response_b'].apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))

df.drop(columns=['prompt_length', 'response_a_length', 'response_b_length'], inplace=True)

print(df.shape)
print(df.head(1).T)

df.to_csv('data/chatbot_arena_conversations.csv', index=False)


Overwriting dataset_format.py


In [8]:
if not os.path.exists('data/chatbot_arena_conversations.csv'):
    !mkdir -p data
    !python dataset_format.py

<div class="alert alert-success" style="color: #333; background-color: #e8fffb; border-color: #bcfff2; border-width: 1px; border-radius: 3px; padding: 10px;">
    
<b>Q1.1.2</b>

<a id="q112"></a>
**Further preprocessing and loading dataset**
- Load the dataset from `data/chatbot_arena_conversations.csv` file in a pandas dataframe.
- Create binary label `label` from the winner columns (`winner_model_a` or `winner_model_b`). 
Note: `winner_model_b==1` => `label=1` => Model b is winner.
- Perform preliminary EDA 
- Display dataset shape, `.head()` and target class ratio.

</div>

In [ ]:
# your code here


<div class="alert alert-success" style="color: #333; background-color: #e8fffb; border-color: #bcfff2; border-width: 1px; border-radius: 3px; padding: 10px;">
    
<b>Q1.1.3 - Filter the dataset</b>

<a id="q113"></a>
Instantiate tokenizer using `AutoTokenizer`. Analyze the token lengths of prompts and responses using the tokenizer. Filter the dataset to remove very long sequences that might cause memory issues during training. Specifically, limit prompt tokens to less than 100 and both response types to less than 400 tokens i.e `response_a` should be less than 400 tokens and `response_b` should also be less than 400 tokens. 
Display number of samples in the dataset with `dataframe.shape`

Tokenizer Settings -
- Be sure to set `tokenizer.pad_token = tokenizer.eos_token`
- Also recommended for good practice  `tokenizer.padding_side = "right"` and `tokenizer.add_eos_token = True`

*Note:* Reason we are limiting the number of tokens - longer texts will require more GPU memory and computation time. Assuming JupyterHub GPU, we believe total maxlen of 1024 tokens per sample should suffice.

</div>

In [ ]:
# your code here


In [ ]:
print("\nDataset shape after filtering:", train.shape)
print("\nLabel distribution after filtering:")
print(train.label.value_counts())

<!-- BEGIN QUESTION -->

<div class="alert alert-success" style="color: #333; background-color: #e8fffb; border-color: #bcfff2; border-width: 1px; border-radius: 3px; padding: 10px;">
<b>Q1.1.4</b>


<a id="q114"></a>
- At this point you should have ~14K samples. We are going to randomly sample (to save on training time) 600 samples of each label type ie. 600 samples with winner='model_a' and 600 samples winner='model_b'. Be sure to set random_state=42.

- Split the dataset into train and validation set with 80/20 split, stratify with label and random_state=42.

- We are also going to sort (ascending=False) the validation set by token length ie. sum prompt_tokens_len, response_a_tokens_len and response_b_tokens_len and then sort.
Why should we sort the validation set ? 

</div>


In [ ]:
# your code here


**Your answer here**

In [ ]:
print("\nTraining set shape:", train_df.shape)
print("Validation set shape:", val_df.shape)
val_df.head()

<div class="alert alert-success" style="color: #333; background-color: #e8fffb; border-color: #bcfff2; border-width: 1px; border-radius: 3px; padding: 10px;">

<b>Q1.2</b>

<a id="q12">**Creating a Tokenized Dataset for Model Training** </a>

Next, you'll create a tokenized dataset using [HuggingFace Datasets](https://huggingface.co/docs/datasets/index) library. To be specific - [from_pandas()](https://huggingface.co/docs/datasets/v3.4.1/en/package_reference/main_classes#datasets.Dataset.from_pandas).

However we cannot train model with raw text, we have to tokenize our text and it must return `input_ids`, `attention_mask` and corresponding `label` for every sample.

Here we provide you with skeleton, you should fill this appropriately.

```python

def tokenize(example, tokenizer):
    # 1. Format the input text with prompt and both responses. You have the freedom to prepend text with for eg: "Prompt:" and/or "Response A:" etc.

    # Combine the prompt and responses into a single text string
    
    # 2. Tokenize the text using the tokenizer
    # Make sure to set appropriate parameters for padding, truncation, etc. Set max_length to 1024
    
    # 3. Extract the input_ids and attention_mask
    
    # 4. Get the label from the example
    
    return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

def load_data(df, tokenizer):
    # 1. Create a Dataset from the pandas DataFrame
    raw_datasets = Dataset.from_pandas(...)
    
    # 2. Apply the tokenize function to each example in the dataset
    # Make sure to:
    # - Pass the tokenizer to the function
    # - Remove original columns that are no longer needed
    # - Handle any other necessary parameters
    tokenized_datasets = raw_datasets.map(
        tokenize,
        ...
    )

    return tokenized_datasets
```

Expected output :

```python
print(tokenized_train)
print(tokenized_val)
```

```python
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 960
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 240
})
```
</div>


In [ ]:
# your code here


In [ ]:
print(tokenized_train)
print(tokenized_val)

<div class="alert alert-success" style="color: #333; background-color: #e8fffb; border-color: #bcfff2; border-width: 1px; border-radius: 3px; padding: 10px;">

<b>Q1.3 Setup LoraConfig and BitsAndBytesConfig </b>

Fine-tuning large language models like Llama 3.2 can be computationally expensive and memory-intensive. To address this, we utilize parameter-efficient fine-tuning techniques such as LoRA (Low-Rank Adaptation), which allows us to update only a small subset of the model's parameters, significantly reducing memory usage and training time while maintaining performance. Additionally, using quantization through BitsAndBytesConfig enables us to load and train the model with reduced precision (e.g., 4-bit or 8-bit), further optimizing memory usage and speeding up computations, especially with limited resources (GPUs.) 

<a id="q13"></a>

**[Lora Config](https://huggingface.co/docs/peft/en/package_reference/lora#peft.LoraConfig)**

We recommend following options -

```python
r=16,
lora_alpha=32,
lora_dropout=0.05,
bias='none',
inference_mode=False,
task_type=TaskType.SEQ_CLS,
target_modules=['q_proj', 'k_proj', 'v_proj',]
```

(a) Ask the following question to [Claude.ai](claude.ai) and [gemini.google.com](https://gemini.google.com/). Note: We are asking two AI models, in case one decides to hallucinate. Also recommended reference documentation - https://huggingface.co/docs/peft/main/en/conceptual_guides/lora

Explain the significance of following parameters - `r`, `lora_alpha` and `target_modules`

Write the answer in your own words - concisely - 3-5 sentences.


(b) **Quantization** - Apply 4-bit quantization using BitsAndBytesConfig. Recommended settings -

```python
load_in_4bit=True,
bnb_4bit_compute_dtype=torch.float16,
bnb_4bit_quant_type="nf4",
```


</div>


In [16]:
# your code here


**Your answer here**

<div class="alert alert-success" style="color: #333; background-color: #e8fffb; border-color: #bcfff2; border-width: 1px; border-radius: 3px; padding: 10px;">

<a id="q14"></a>
**1.4 Custom Classification Head for LLaMA**

In this question, you’ll implement a custom classification head for the pre-trained LLaMA model. Your task is to complete the implementation of the classifier by subclassing LlamaPreTrainedModel.

Your custom model should:
- Accept inputs (input_ids, attention_mask, labels, etc) and pass them through the pre-trained LLaMA model [LlamaModel](https://huggingface.co/docs/transformers/v4.49.0/en/model_doc/llama#transformers.LlamaModel).
- Extract embeddings from the last token of the sequence (based on attention mask).
- Classify embeddings into two labels using a linear layer `nn.Linear`.
- Output predictions using the Hugging Face standard SequenceClassifierOutput.

- Your class will resemble [LlamaForSequenceClassification](https://github.com/huggingface/transformers/blob/a22a4378d97d06b7a1d9abad6e0086d30fdea199/src/transformers/models/llama/modeling_llama.py#L893)

- We provide you with the loss function below.


Your implementation should match the following structure:

```python

class CS1090B_LlamaForClassification(LlamaPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)

        # YOUR CODE HERE
        pass
    
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        position_ids=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    )

        # YOUR CODE HERE


        # Calculate loss
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return SequenceClassifierOutput(
            loss=loss,
            logits=pooled_logits,
            hidden_states=outputs.hidden_states if output_hidden_states else None,
            attentions=outputs.attentions if output_attentions else None,
        )

```

</div>

In [17]:
# your code here


In [18]:
## Test your model
if 0:
    model = CS1090B_LlamaForClassification.from_pretrained(
            model_path,
            quantization_config=quantization_config,
        )

    text = "Hello CS1090B!!"
    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding="max_length",
        max_length=16,
        truncation=True
    )

    outputs = model(
            input_ids=inputs["input_ids"].to(device),
            attention_mask=inputs["attention_mask"].to(device)
        )
    print(outputs.logits)

    del model, outputs, inputs, text
    gc.collect()
    torch.cuda.empty_cache()

<div class="alert alert-success" style="color: #333; background-color: #e8fffb; border-color: #bcfff2; border-width: 1px; border-radius: 3px; padding: 10px;">

<a id="q15">**Q1.5 Metric and Training** </a>


<b>Q1.5.1 - Implementing Evaluation Metrics</b>

<a id="q151"></a>

Next, we are going to customize [`compute_metrics()`](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.compute_metrics)` which will be utilized when we train our model.

Implement a `compute_metrics` function that takes evaluation predictions and calculates performance metrics. This function should:
- Take evaluation predictions as input (containing predictions and labels)
- Apply softmax to the raw prediction logits
- Calculate accuracy by comparing the predicted labels to the true labels
- Calculate log loss (cross-entropy) between the predicted probabilities and true labels
- Return a dictionary containing both metrics


```python
def compute_metrics(eval_pred):
    # Your code here

    return {
            "accuracy": accuracy,
            "log_loss": loss
        }
```


</div>

In [19]:
# your code here


In [ ]:
## Test your compute_metrics function

# Simulate predictions (logits) and true labels
num_samples = 5
num_classes = 2
np.random.seed(42)
logits = np.random.randn(num_samples, num_classes)
labels = np.random.randint(0, num_classes, size=num_samples)

# Create a mock object to simulate eval_pred
EvalPred = namedtuple("EvalPred", ["predictions", "label_ids"])
eval_pred = EvalPred(predictions=logits, label_ids=labels)

print(compute_metrics(eval_pred))
del num_samples, num_classes, logits, labels, eval_pred

<div class="alert alert-success" style="color: #333; background-color: #e8fffb; border-color: #bcfff2; border-width: 1px; border-radius: 3px; padding: 10px;">

<b>Q1.5.2 - Model Training </b>

<a id="q152"></a>


Implement a complete training pipeline for the LLaMA classification model using the HuggingFace Trainer API. Your implementation should:

 - Load and prepare the tokenized datasets created earlier i.e. call `load_data()`
 - Initialize the CS1090B_LlamaForClassification model with the appropriate quantization configuration
 - Prepare the model for k-bit training [Link](https://huggingface.co/docs/peft/v0.15.0/en/package_reference/peft_model#peft.prepare_model_for_kbit_training). Recommended setting `use_gradient_checkpointing=False`
 - Apply the LoRA adapters using the configuration from Q1.3 [Link](https://huggingface.co/docs/peft/v0.15.0/en/package_reference/peft_model#peft.get_peft_model)
 - Print number of trainable parameters - `model.print_trainable_parameters()`
 - Configure training arguments with appropriate hyperparameters (learning rate, batch size, etc.) [Link](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments)

```python
 Recommended parameters -

 training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=1,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=4,
        warmup_steps=50,
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        push_to_hub=False,
        learning_rate=1e-4,
        report_to="none",
    )

```

 - Create a data collator for padding [Link](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding)

 - Initialize the Trainer with the model, training arguments, tokenizer, datasets, data collator and evaluation metrics.
 - Execute the training process
 - Save the trained model
 - Evaluate the model on the validation set and report the results. Validation accuracy should be above 0.6.

 Note: Pay careful attention to batch sizes and gradient checkpointing settings, as these can significantly impact training time and memory usage. Approx. expected training time on JuputerHub ~35mins.

</div>

In [ ]:
# your code here


<div class="alert alert-success" style="color: #333; background-color: #e8fffb; border-color: #bcfff2; border-width: 1px; border-radius: 3px; padding: 10px;">

<b>Q1.6 - Predict and Display Confusion Matrix </b>

<a id="q16"></a>

Run the cell below to display confusion matrix on validation set
</div>

In [ ]:
# `tokenized_val` is a HuggingFace Dataset object
output = trainer.predict(tokenized_val)
logits = output.predictions  
print(logits.shape)

predicted_classes = np.argmax(logits, axis=1)

true_labels = output.label_ids


cm = confusion_matrix(true_labels, predicted_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap="Blues")
plt.title("Confusion Matrix on Validation Set")
plt.show()


In [ ]:
time_end = time.time()
print(f"It took {(time_end - time_start)/60:.2f} minutes for this notebook to run")

**This concludes Part 1 👏**

**Please continue to Parts 2 & 3 inthe second notebook**